In [1]:
import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import numpy as np
from quantfreedom.nb.simulate import simulate_up_to_6
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.evaluators.evaluators import eval_is_below
from quantfreedom.plotting.strat_dash import strat_dashboard

np.set_printoptions(formatter={'float_kind':"{:.2f}".format})

pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv(
    'E:/Coding/backtesters/QuantFreedom/tests/data/30min.csv', index_col='time')

rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=np.arange(15, 61, 15),
)

rsi_eval = eval_is_below(
    rsi_ind,
    np.arange(40, 61, 10),
)


In [2]:
final_array, order_records = simulate_up_to_6(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=rsi_eval[45].values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=4,
    risk_rewards=[3,5,6],
    size_pct=1.,
    # sl_pcts=[2,3,4],
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(2,5,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,4,1),
    tsl_when_pct_from_avg_entry=np.arange(1,4,1),
)


In [4]:
strat_dashboard(
    prices=prices,
    eval_results=rsi_eval[45][50],
    order_records=order_records[order_records['settings_id'] == 1],
    talib_indicator=rsi_ind[45],
    
)

Dash is running on http://127.0.0.1:3003/

Dash app running on http://127.0.0.1:3003/
